In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [9]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [10]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [11]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 408/408 [00:00<00:00, 6933.38 examples/s]


In [12]:
from transformers import TrainingArguments


In [13]:
training_args = TrainingArguments("test-trainer")

In [6]:
from transformers import AutoModelForSequenceClassification


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

/tmp/ipykernel_57669/2812354321.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()

 36%|███▋      | 500/1377 [17:17<28:25,  1.94s/it]  

{'loss': 0.4942, 'grad_norm': 35.4779052734375, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


 73%|███████▎  | 1000/1377 [33:13<11:40,  1.86s/it]

{'loss': 0.2545, 'grad_norm': 0.19076409935951233, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


100%|██████████| 1377/1377 [47:49<00:00,  2.08s/it]

{'train_runtime': 2869.9581, 'train_samples_per_second': 3.834, 'train_steps_per_second': 0.48, 'train_loss': 0.3061215934670309, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3061215934670309, metrics={'train_runtime': 2869.9581, 'train_samples_per_second': 3.834, 'train_steps_per_second': 0.48, 'total_flos': 405114969714960.0, 'train_loss': 0.3061215934670309, 'epoch': 3.0})

In [22]:
predictions = trainer.predict(tokenized_datasets["validation"])

100%|██████████| 51/51 [00:35<00:00,  1.43it/s]


In [23]:
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [24]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [27]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8431372549019608, 'f1': 0.8896551724137931}